# SETUP

The first part of the code sets up the pca_wavelet network, the training comes later. Most of this code comes from the original authors

In [2]:
import sys
sys.path.append('../segmentation_helper')

import tensorflow as tf
import keras
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import data_loader as dl
import model_broker as mb
import os

GPU device not found
Found GPU at: 


In [3]:
tf.keras.backend.set_floatx("float64")

In [ ]:
loader = dl.DataLoader(IMAGE_SIZE=128)
img_train,img_test = loader.import_processed_img()
seg_train,seg_test = loader.import_processed_seg()
combined_train, combined_test = loader.import_processed_combined()

In [ ]:
plt.imshow((next(iter(seg_train))[0]))

In [ ]:
broker = mb.ModelBroker()
seghead,seginvhead = broker.build_model(seg_train,seg_test,dirname="bird_seg",keep_percent=0.3,count=2,samplesize=100)
seghead,seginvhead = broker.load_model(seg_train,seg_test,dirname="bird_seg",keep_percent=0.3,count=2)
imghead,imginvhead = broker.build_model(img_train,img_test,dirname="bird_img",keep_percent=0.3,count=2,samplesize=100)
imghead,imginvhead = broker.load_model(img_train,img_test,dirname="bird_img",keep_percent=0.3,count=2)

In [ ]:
broker.check_build(seghead,seginvhead,seg_train)

In [ ]:
broker.check_build(imghead,imginvhead,img_train)

# Training

This is the loop used to find the variables to find A and b in y=Ax+b

In [ ]:
shape = imghead(next(iter(img_train))).shape

In [ ]:
shape_0 = shape[3]
shape_1 = shape[1]*shape[2]
shape_2 = shape[1]

In [ ]:
n = 0.0


xxt = np.zeros([shape_0,shape_0])
yxt = np.zeros([shape_0,shape_0])
x = np.ones([shape_1])
x_m = np.zeros([shape_0])
y = np.ones([shape_1]) 
y_m = np.zeros([shape_0])

bar = tqdm.notebook.tqdm(total = int(combined_train.cardinality()))

for item in iter(combined_train):
    bar.update(1)
    image = item[0][0]
    segmentation = item[0][1]
    
    imgdecom = imghead(image)
    segdecom = seghead(segmentation)
    
    mat = tf.reshape(imgdecom,[-1,imgdecom.shape[-1]])
    segmat = tf.reshape(segdecom,[-1,imgdecom.shape[-1]])
    
    cov = tf.tensordot(mat,mat,[0,0])
    xxt += cov
    #del cov
    
    segcov = tf.tensordot(mat,segmat,[0,0])
    yxt += segcov
    #del segcov
    
    x_m += tf.linalg.matvec(mat,x,transpose_a=True)
    y_m += tf.linalg.matvec(segmat,y,transpose_a=True)
    
    n += 1

In [ ]:
directory = "pca_vars/"+"seg2img"
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
np.save(directory+"/xxt",xxt)
np.save(directory+"/yxt",yxt)
np.save(directory+"/x_m",x_m)
np.save(directory+"/y_m",y_m)
np.save(directory+"/n",n)

## Calculating A and B

This section uses the values found in the training loop to calculate values for A and B

In [ ]:
xxt = np.load(directory+"/xxt.npy")
yxt = np.load(directory+"/yxt.npy")
x_m = np.load(directory+"/x_m.npy")
y_m = np.load(directory+"/y_m.npy")
n = np.load(directory+"/n.npy")

In [ ]:
xxt = (xxt - tf.tensordot(x_m,x_m,0))/n
yxt = (yxt - tf.tensordot(x_m,y_m,0))/n

In [ ]:
inverse_xxt = tf.linalg.pinv(xxt)

In [ ]:
A = tf.reduce_sum(tf.tensordot(inverse_xxt,yxt,[0,0]),0)

In [ ]:
b = (y_m - A*x_m)/n

In [ ]:
img_train, img_test = loader.import_processed_img()

In [ ]:
image,seg_base = next(iter(combined_test.shuffle(100)))[0]
imgdecom = imghead(image)
imgdecom = tf.reshape(imgdecom,(-1,shape_0))
y = A*imgdecom + b
seg = seginvhead(tf.reshape(y,(1,shape_2,shape_2,shape_0)))
plt.imshow(np.hstack([image,tf.round(seg_base),tf.round(seg[0])]))